# Анализ источников новостей

Цель данного исследования: проанализировать источники новостных лент, сравнить рубрики, подумать над возможностью добавление\удаление категорий для проекта "Радар новостей рунета"

In [32]:
import numpy as np
import pandas as pd
import datetime as DT

## RT

Загрузим датасет rt. Одновременно сделаем преобразование даты из строки

In [37]:
dateparse = lambda x: pd.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

In [151]:
data_rt = pd.read_csv('../data/rt.csv',
                      sep=',',
                      index_col=False,
                      escapechar='\\',
                      parse_dates=['date'],
                      date_parser=dateparse,
                      quotechar='\"')

In [152]:
# Проверим преобразование
data_rt.dtypes

date           datetime64[ns]
url                    object
edition                object
topics                 object
authors               float64
title                  object
text                   object
reposts_fb             object
reposts_vk             object
reposts_ok             object
reposts_twi            object
reposts_lj             object
reposts_tg             object
likes                  object
views                  object
comm_count             object
dtype: object

In [153]:
data_rt.shape

(106644, 16)

In [154]:
# Так выглядет датасет RT
data_rt.head()

,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2019-06-26 14:00:00,https://russian.rt.com/russia/news/644595-avto...,-,Россия,NaN,Эксперт прокомментировал ситуацию с возможным ...,"Как считает эксперт, россиянам не стоит пережи...",-,-,-,-,-,-,-,-,-
1,2019-06-26 19:56:00,https://russian.rt.com/sport/article/644740-nu...,-,Спорт,NaN,"Встреча Нурмагомедова с Макгрегором, восстанов...",«Конор может посетить поединок Хабиба с Порье»...,-,-,-,-,-,-,-,-,-
2,2019-06-26 14:52:00,https://russian.rt.com/russia/news/644619-puti...,-,Россия,NaN,Путин подарил Зюганову на юбилей материалы съе...,"Открывая заседание Госсовета в Кремле, Путин п...",-,-,-,-,-,-,-,-,-
3,2019-06-26 14:44:00,https://russian.rt.com/world/news/644618-parla...,-,Мир,NaN,"Парламентарии США, ФРГ и России проведут встре...",Об этом информируетnРИА Новостиnсо ссылкой на ...,-,-,-,-,-,-,-,-,-
4,2019-06-26 14:42:00,https://russian.rt.com/sport/news/644617-mok-a...,-,Спорт,NaN,МОК принял решение лишить признания AIBA,Решением 134-й сессии МОК в швейцарской Лозанн...,-,-,-,-,-,-,-,-,-


In [155]:
# Выясним в каким столбцах есть пустые данные
data_rt.isnull().any()

date           False
url            False
edition        False
topics         False
authors         True
title          False
text            True
reposts_fb     False
reposts_vk     False
reposts_ok     False
reposts_twi    False
reposts_lj     False
reposts_tg     False
likes          False
views          False
comm_count     False
dtype: bool

Как видим, в некоторых столбцах вместо значения Nan используется "-", их преобразованием займемся позже. А пока посмотрим строки, в которых пустые тексты новостей

In [156]:
print('Кол-во строк с значением NaN в столбце "Text" = ', data_rt['text'].isnull().sum())
data_rt[data_rt['text'].isnull()]

Кол-во строк с значением NaN в столбце "Text" =  12


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
37474,2019-06-05 13:47:00,https://russian.rt.com/russia/video/638188-obr...,-,Россия,NaN,\n В Дагестане люди пытаются вручну...,NaN,-,-,-,-,-,-,-,-,-
38249,2017-12-11 19:32:00,https://russian.rt.com/sport/article/459073-ro...,-,Спорт,NaN,Не без труда: Россия в овертайме победила Южну...,NaN,-,-,-,-,-,-,-,-,-
43956,2017-12-08 01:29:00,https://russian.rt.com/world/article/457555-la...,-,Мир,NaN,"Будущее Сирии, перенос столицы Израиля и ситуа...",NaN,-,-,-,-,-,-,-,-,-
84132,2016-07-29 10:08:01,https://russian.rt.com/article/314450-dollar-p...,-,Россия,NaN,Доллар превысил 67 рублей,NaN,-,-,-,-,-,-,-,-,-
86097,2016-07-27 11:24:00,https://russian.rt.com/article/314142-sbornaya...,-,Спорт,NaN,Сборная России по тхэквондо в полном составе д...,NaN,-,-,-,-,-,-,-,-,-
86103,2016-07-27 08:56:37,https://russian.rt.com/article/314121-cb-rf-ot...,-,Россия,NaN,ЦБ РФ отозвал лицензию у банка «БФГ-Кредит»,NaN,-,-,-,-,-,-,-,-,-
92836,2016-07-19 17:43:00,https://russian.rt.com/article/312972-mok-inic...,-,Спорт,NaN,МОК инициирует повторный анализ проб российски...,NaN,-,-,-,-,-,-,-,-,-
93872,2017-11-09 18:22:00,https://russian.rt.com/world/article/447764-ss...,-,Мир,NaN,«Это могла быть только внутренняя утечка»: экс...,NaN,-,-,-,-,-,-,-,-,-
93917,2016-07-18 16:47:26,https://russian.rt.com/article/312776-otchyot-...,-,Спорт,NaN,"Отчёт WADA: Метод, используемый ФСБ для вскрыт...",NaN,-,-,-,-,-,-,-,-,-
93919,2016-07-18 16:25:00,https://russian.rt.com/article/312771-otchyot-...,-,Спорт,NaN,Отчёт WADA: Минспорт РФ и ФСБ участвовали в за...,NaN,-,-,-,-,-,-,-,-,-


In [157]:
#Удалим эти строки
data_rt['text'].dropna(inplace=True)

In [158]:
print('Кол-во строк с значением NaN в столбце "Text" = ', data_rt['text'].isnull().sum())

Кол-во строк с значением NaN в столбце "Text" =  0


Посмотрим сколько элементов Nan в столбце авторов

In [166]:
print('Кол-во строк с значением NaN в столбце "authors" = ', data_rt['authors'].isnull().sum())

Кол-во строк с значением NaN в столбце "authors" =  106644


Как видим все значения авторов Nan. Можно не использовать данные столбец

Посмотрим, что в столбцах, которые содержат дефис, как обозначение пустых данных

In [211]:
print('edition: ',data_rt['edition'].unique())
print('reposts_fb: ', data_rt['reposts_fb'].unique())
print('reposts_vk: ',data_rt['reposts_vk'].unique())
print('reposts_ok: ',data_rt['reposts_ok'].unique())
print('reposts_twi: ',data_rt['reposts_twi'].unique())
print('reposts_lj: ',data_rt['reposts_lj'].unique())
print('reposts_tg: ',data_rt['reposts_tg'].unique())
print('likes: ',data_rt['likes'].unique())
print('views: ',data_rt['views'].unique())
print('comm_count: ',data_rt['comm_count'].unique())

edition:  ['-']
reposts_fb:  ['-']
reposts_vk:  ['-']
reposts_ok:  ['-']
reposts_twi:  ['-']
reposts_lj:  ['-']
reposts_tg:  ['-']
likes:  ['-']
views:  ['-']
comm_count:  ['-']


Значений нет, можно удалять

### Рубрики RT

In [162]:
# Считаем количество рубрик в данном датасете, а также выведем значения
print('Количество рубрик = ', data_rt.topics.unique().shape)
data_rt['topics'].unique()

Количество рубрик =  (10,)


array(['Россия', 'Спорт', 'Мир', 'Без политики', 'Новости партнёров',
       'Бывший СССР', 'Экономика', 'Наука', 'Мероприятия RT',
       'Пресс-релизы'], dtype=object)

Все рубрики, с количеством новостей в каждой группе

In [172]:
data_rt.groupby('topics')['topics'].count()

topics
Без политики          4489
Бывший СССР          10078
Мероприятия RT          13
Мир                  26476
Наука                 1496
Новости партнёров       14
Пресс-релизы            92
Россия               32963
Спорт                26702
Экономика             4321
Name: topics, dtype: int64

## ria

In [212]:
dateparse = lambda x: pd.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

In [215]:
data_ria = pd.read_csv('../data/ria.csv',
                      sep=',',
                      index_col=False,
                      escapechar='\\',
                      parse_dates=['date'],
                      date_parser=dateparse,
                      quotechar='\"', error_bad_lines=False)

b'Skipping line 49866: expected 17 fields, saw 33\n'


In [217]:
# Проверим преобразование
data_ria.dtypes

date           datetime64[ns]
url                    object
edition                object
topics                 object
authors               float64
title                  object
text                   object
reposts_fb             object
reposts_vk             object
reposts_ok             object
reposts_twi            object
reposts_lj             object
reposts_tg             object
likes                   int64
views                   int64
comm_count             object
dtype: object

In [219]:
data_ria.shape

(101543, 16)

In [220]:
# Так выглядет датасет ria
data_ria.head()

,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2019-08-10 06:35:00,https://ria.ru/20190810/1557366973.html,ria.ru,Происшествия,NaN,Восемнадцать туристов застряли на пляже в Прим...,"ВЛАДИВОСТОК, 10 авг – РИА Новости.nВосемнадцат...",-,-,-,-,-,-,0,70,-
1,2019-08-10 03:25:00,https://ria.ru/20190810/1557366343.html,ria.ru,В мире,NaN,Президент Парагвая даст показания по делу о сп...,"БУЭНОС-АЙРЕС, 10 авг – РИА Новости.nПрезидент ...",-,-,-,-,-,-,1,177,-
2,2019-08-10 03:26:00,https://ria.ru/20190810/1557366366.html,ria.ru,В мире,NaN,Андрейченко обвинили в угрозе теракта после но...,"МОСКВА, 10 авг - РИА Новости.nДмитрию Андрейче...",-,-,-,-,-,-,2,919,-
3,2019-08-10 03:31:00,https://ria.ru/20190810/1557366398.html,ria.ru,В мире,NaN,"Япония заявила, что не испугалась субботних пу...","МОСКВА, 10 авг - РИА Новости.nПравительство Яп...",-,-,-,-,-,-,1,140,-
4,2019-08-10 03:32:00,https://ria.ru/20190810/1557366421.html,ria.ru,Происшествия,NaN,"Власти Якутии погасят долг перед жителями, уча...","ЯКУТСК, 10 авг – РИА Новости.nСредства резервн...",-,-,-,-,-,-,3,1196,-


In [221]:
# Выясним в каким столбцах есть пустые данные
data_ria.isnull().any()

date           False
url            False
edition        False
topics         False
authors         True
title          False
text            True
reposts_fb     False
reposts_vk     False
reposts_ok     False
reposts_twi    False
reposts_lj     False
reposts_tg     False
likes          False
views          False
comm_count     False
dtype: bool

Как видим, в некоторых столбцах вместо значения Nan используется "-", их преобразованием займемся позже. А пока посмотрим строки, в которых пустые тексты новостей

In [223]:
print('Кол-во строк с значением NaN в столбце "Text" = ', data_ria['text'].isnull().sum())
data_ria[data_ria['text'].isnull()]

Кол-во строк с значением NaN в столбце "Text" =  2


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
45983,2019-02-26 14:22:00,https://ria.ru/20190226/1551377894.html,ria.ru,Общество,NaN,Самый красивый трамвайный маршрут России,NaN,-,-,-,-,-,-,3,1140,-
97961,2019-07-29 16:15:00,https://ria.ru/20190729/1556918528.html,ria.ru,Безопасность,NaN,Крылья России. Самолёты ОКБ Сухого,NaN,-,-,-,-,-,-,43,1383,-


In [224]:
#Всего две новости, удалим эти строки
data_ria['text'].dropna(inplace=True)

In [225]:
print('Кол-во строк с значением NaN в столбце "Text" = ', data_ria['text'].isnull().sum())

Кол-во строк с значением NaN в столбце "Text" =  0


Посмотрим сколько элементов Nan в столбце авторов

In [226]:
print('Кол-во строк с значением NaN в столбце "authors" = ', data_ria['authors'].isnull().sum())

Кол-во строк с значением NaN в столбце "authors" =  101543


Как видим все значения авторов Nan. Можно не использовать данные столбец

Посмотрим, что в столбцах, которые содержат дефис, как обозначение пустых данных

In [227]:
print('edition: ',data_ria['edition'].unique())
print('reposts_fb: ', data_ria['reposts_fb'].unique())
print('reposts_vk: ',data_ria['reposts_vk'].unique())
print('reposts_ok: ',data_ria['reposts_ok'].unique())
print('reposts_twi: ',data_ria['reposts_twi'].unique())
print('reposts_lj: ',data_ria['reposts_lj'].unique())
print('reposts_tg: ',data_ria['reposts_tg'].unique())
print('likes: ',data_ria['likes'].unique())
print('views: ',data_ria['views'].unique())
print('comm_count: ',data_ria['comm_count'].unique())

edition:  ['ria.ru']
reposts_fb:  ['-']
reposts_vk:  ['-']
reposts_ok:  ['-']
reposts_twi:  ['-']
reposts_lj:  ['-']
reposts_tg:  ['-']
likes:  [    0     1     2     3    14    63    19     5    16   151     6    11
     4    29    13   184     8     7    10    30    28    18   108    95
   239    36    24   333   257    51    39     9    88    77    27    46
    12    70    23    22    34    67   100   421   256   381   318    33
    55   141    32   189    64    15   619    45    20   165    54   646
    82   160    81   250   216    17    50    31    47   161   366    75
   114    25   245   434    53    21   392   109   628   674   835   133
   402    43    59    52    26   113   219   474    74   289    83   397
   375    68   315    56   564   128   356    80    49   106   195  1043
   111    94    73   755   238   470    84    69   607   115   139    57
   145    62    35   155    44    37    60    79   263    71   362   104
    42   231    58  1502    99  1971   188   267   66

В edition, likes и views данные есть, остальные можно удалять

### Рубрики Ria

In [228]:
# Считаем количество рубрик в датасете Ria, а также выведем значения
print('Количество рубрик = ', data_ria.topics.unique().shape)
data_ria['topics'].unique()

Количество рубрик =  (1120,)


array(['Происшествия', 'В мире', 'Экономика', ...,
       'ЧП на военном складе в Красноярском крае', 'Лиссабон (город)',
       'The Guardian'], dtype=object)

In [229]:
data_ria.groupby('topics')['topics'].count()

topics
2 февраля - день сурка      1
BAFTA                       1
BBC                         8
Cirque du Soleil            1
Comedy Club Production      1
                         ... 
пассажиры                   1
психология                  1
религия                   192
самолеты                    2
туристы                    21
Name: topics, Length: 1120, dtype: int64

Тут надо разбираться, все ли рубрики нам нужны